# Basic Pushbroom Imager on a Satellite

Computing the basic parameters of a satellite pushbroom imager. Valid for a single band.

Reference parameters for the scene and camera position with respect to the target are given below.

In [32]:
from pint import UnitRegistry
import numpy as np

# Init units
u = UnitRegistry()

# sat positional params
# ---------------------
sat_altitude = 540.0 * u.km
ground_vel = 6998.1 * u.m / u.s

# converted to generic params
distance = sat_altitude
target_rel_velocity = ground_vel

# scene ref params
# ----------------

# blue: 450-485 nm
# green: 500-565 nm
# red: 625-750 nm
# nir: 780-2500 nm
ref_wavelength = 632 * u.nm


Optical Parameters are given below.

In [33]:
# optical params (design)
# -----------------------

focal_length = 580 * u.mm
aperture_diam = 95 * u.mm
# image diameter on the focal plane (fixed by the design)
image_diam_on_focal_plane = 30 * u.mm 


# optical params (computed)
# -------------------------

f_nr = focal_length / aperture_diam
full_fov = 2 * np.arctan((image_diam_on_focal_plane/2.) / focal_length)
aperture_area = np.pi * (aperture_diam/2.)**2
# solid angle = 2pi h/r
aperture_solid_angle = np.pi / (focal_length / (aperture_diam/2.))**2 * u.steradian

u.define('lp = 1 * dimensionless = lp')
spatial_cutoff_freq = (1. * u.lp/(ref_wavelength*f_nr).to(u.mm)) # perfect incoherent optics

print(f"F-number : {f_nr:~.4}")
print(f"Full FoV : {full_fov.to(u.deg):~.4}")

print(f"Spatial cut-off frequency : {spatial_cutoff_freq:~.4P}")

F-number : 6.105
Full FoV : 2.963 deg
Spatial cut-off frequency : 259.2 lp/mm


Detector Parameters are given here

In [34]:
# Detector Params (design)
# ------------------------

pix_pitch = 5.5 * u.um
horiz_pixels_used = 4096
horiz_pixels = 4096 
vert_pixels_used = 3072
vert_pixels = 3072

binning = 1 # should be int
tdi_stages = 2 # should be int, TDI is with binning

full_well_capacity = 13500 * u.e
dark_current = 70 * u.e/u.s
temporal_dark_noise = 13. * u.e

# frame duration that cannot be used for inetgration or imaging
frame_overhead_time = 35 * u.us 
# time between frame durations that can be used for inetgration or imaging
overlap_time = 15 * u.us


# Detector Params (computed)
# --------------------------

nyq_freq_native = 1 * u.lp/(2*pix_pitch).to(u.mm)
nyq_freq = 1 * u.lp/(2*binning*pix_pitch).to(u.mm)

if binning==1:  
    print(f"Nyquist Frequency : {nyq_freq_native:~.4P}")
else:
    print(f"Nyquist Frequency (native) : {nyq_freq_native:~.4P}")
    print(f"Nyquist Frequency (binned) : {nyq_freq:~.4P}")

print(f"Full well capacity : {full_well_capacity:~P}")

frame_pixel_count_used = horiz_pixels_used * vert_pixels_used * u.pixel
print(f"Number of pixels (used) : {frame_pixel_count_used.to("Mpixel"):~.4}")

# Instantaneous Field of View (works in vertical and horizontal)
ifov_native = 2*np.arctan( (pix_pitch / 2.) /focal_length)
ifov = 2*np.arctan( (pix_pitch * binning / 2.) /focal_length)

if binning==1:  
    print(f"IFoV : {ifov_native.to(u.mdeg):~.4P}  ({ifov_native.to(u.urad):~.4P})")
else:
    print(f"IFoV (native) : {ifov_native.to(u.mdeg):~.4P}  ({ifov_native.to(u.urad):~.4P})")
    print(f"IFoV (binned) : {ifov.to(u.mdeg):~.4P}  ({ifov.to(u.urad):~.4P})")

# pixel physical area
pix_area = pix_pitch**2

# Pixel solid angle (of a pyramid)
pix_solid_angle_native = 4 * np.arcsin( np.sin(ifov_native/2.0) * np.sin(ifov_native/2.0) ) 
pix_solid_angle_native = (pix_solid_angle_native * u.rad).to(u.steradian) # correct the unit from rad to sr

pix_solid_angle = 4 * np.arcsin( np.sin(ifov/2.0) * np.sin(ifov/2.0) ) 
pix_solid_angle = (pix_solid_angle * u.rad).to(u.steradian) # correct the unit from rad to sr

if binning==1: 
    print(f"Pixel solid angle : {pix_solid_angle_native:~.4P}")
else:
    print(f"Pixel solid angle (native) : {pix_solid_angle_native:~.4P}")
    print(f"Pixel solid angle (binned) : {pix_solid_angle:~.4P}")

# Full FoVs
horiz_fov = 2*np.tan(ifov_native * horiz_pixels_used /2.)
vertical_fov = 2*np.tan(ifov_native * vert_pixels_used /2.)

print(f"Horizontal Full FoV : {horiz_fov.to(u.deg):~.4P} (used pixels only)")
print(f"Vertical Full FoV   : {vertical_fov.to(u.deg):~.4P} (used pixels only)")


Nyquist Frequency : 90.91 lp/mm
Full well capacity : 13500 e
Number of pixels (used) : 12.58 Mpixel
IFoV : 0.5433 mdeg  (9.483 µrad)
Pixel solid angle : 8.992×10⁻¹¹ sr
Horizontal Full FoV : 2.226 deg (used pixels only)
Vertical Full FoV   : 1.669 deg (used pixels only)


Geometric Output Parameters

(Fwd Motion Compensation not implemented)

In [35]:
# Geometric Output Parameters (computed)
# --------------------------------------

# Ground sample distance at nadir
spatial_sample_distance_native = (distance*(pix_pitch/focal_length)).to_reduced_units().to(u.m)
spatial_sample_distance = (distance*(pix_pitch * binning/focal_length)).to_reduced_units().to(u.m)

# swath assuming flat plate and constant Instantaneous FoV
swath = 2*np.tan(ifov_native * horiz_pixels_used /2.) * distance

if binning==1: 
    print(f"Spatial Sample Distance : {spatial_sample_distance_native:~.4P}  (@ nadir)")
else:
    print(f"Spatial Sample Distance (native) : {spatial_sample_distance_native:~.4P}  (@ nadir)")
    print(f"Spatial Sample Distance (binned) : {spatial_sample_distance:~.4P}  (@ nadir)")

print(f"Swath : {swath.to(u.km):~.4P}  (disregarding Earth curvature)") 


Spatial Sample Distance : 5.121 m  (@ nadir)
Swath : 20.98 km  (disregarding Earth curvature)


Timings

Note the user defined value integration duration (though limited by the computed max integration duration).

In [36]:
# Timings
# -------

line_duration_native = spatial_sample_distance_native / target_rel_velocity # square pixels
line_duration = spatial_sample_distance / target_rel_velocity # square pixels

if binning==1: 
    print(f"Line duration : {line_duration_native.to(u.ms):~.4P} ({(1/line_duration_native).to("Hz"):~.6P} line rate (native)) (square pix)")
else:
    print(f"Line duration (native) : {line_duration_native.to(u.ms):~.4P} ({(1/line_duration_native).to("Hz"):~.6P} line rate (native)) (square pix)")
    print(f"Line duration (binned) : {line_duration.to(u.ms):~.4P} ({(1/line_duration).to("Hz"):~.6P} line rate) (square pix)")

# native line max integration duration possible
max_integ_duration = line_duration_native - frame_overhead_time + overlap_time
# acutal integration duration (<= max_integ_duration) set by the user, in this example as a percentage
integ_duration = max_integ_duration * 0.4 

# Total TDI column duration
tdi_col_duration = line_duration * tdi_stages

print(f"Max integ duration {'' if binning==1 else '(native)'}: {max_integ_duration.to(u.ms):~.4P}")
print(f"Actual integ duration {'' if binning==1 else '(native)'}: {integ_duration.to(u.ms):~.4P} ({(integ_duration/max_integ_duration).m:.1%} of max)")

print(f"Total TDI column duration : {tdi_col_duration.to(u.ms):~.4P} ({tdi_stages}x)")


Line duration : 0.7317 ms (1366.63 Hz line rate (native)) (square pix)
Max integ duration : 0.7117 ms
Actual integ duration : 0.2847 ms (40.0% of max)
Total TDI column duration : 1.463 ms (2x)


Readout electronics

In [37]:
# Read-out Electronics Params (design)
# ------------------------------------

pixel_encoding = 12 * u.bpp

# print(f"Pixel Encoding : {pixel_encoding}")

# data overhead during readout and write
data_write_overhead = 2./100.

# compression
compression_on = False 
compression_ratio = 2.4


# Read-out Electronics Params (computed)
# --------------------------------------

print(f"Read rate {'' if binning==1 else '(native)'}: {(horiz_pixels * u.pixel/line_duration_native).to('Mpixel/s'):~.5P}")

pixel_read_datarate_native= horiz_pixels * u.pixel * pixel_encoding / line_duration_native
pixel_read_datarate = horiz_pixels * u.pixel * pixel_encoding / line_duration # with binning

if binning==1: 
    print(f"Read datarate : {pixel_read_datarate_native.to('Mbit/s'):~.4P}")
else:
    print(f"Read datarate (native) : {pixel_read_datarate_native.to('Mbit/s'):~.4P}")
    print(f"Read datarate (binned) : {pixel_read_datarate.to('Mbit/s'):~.4P}")

# init write datarate (with data compression if applicable)
if compression_on:
    pixel_write_datarate_native = pixel_read_datarate_native / compression_ratio
    pixel_write_datarate = pixel_read_datarate / compression_ratio
else:
    pixel_write_datarate_native = pixel_read_datarate_native
    pixel_write_datarate = pixel_read_datarate 

pixel_write_datarate_native = pixel_write_datarate_native * (1+data_write_overhead)
pixel_write_datarate = pixel_write_datarate * (1.+data_write_overhead)

if binning==1: 
    print(f"Write datarate : {pixel_write_datarate_native.to('Mbit/s'):~.4P}  (after overheads{' and compression' if compression_on else ''})")
else:
    print(f"Write datarate (native) : {pixel_write_datarate_native.to('Mbit/s'):~.4P}  (after overheads{' and compression' if compression_on else ''}")
    print(f"Write datarate (binned) : {pixel_write_datarate.to('Mbit/s'):~.4P}  (after overheads{' and compression' if compression_on else ''}")


Read rate : 5.5977 Mpixel/s
Read datarate : 67.17 Mbit/s
Write datarate : 68.52 Mbit/s  (after overheads)
